Use a simple model to train and test the data

2017-01-03

---

In [1]:
import pandas as pd

---

#### Prepare train/test

##### Train

In [7]:
train_df = pd.read_csv('./data/data_format1/train_format1.csv')
train_df.shape

(260864, 3)

In [8]:
train_df.head(2)

,user_id,merchant_id,label
0,34176,3906,0
1,34176,121,0


In [13]:
merchant_id_arr = train_df.merchant_id.unique()
merchant_id_arr.size

1993

##### Test

In [11]:
test_df = pd.read_csv('./data/data_format1/test_format1.csv')
test_df.shape

(261477, 3)

In [12]:
test_df.head(2)

,user_id,merchant_id,prob
0,163968,4605,NaN
1,360576,1581,NaN


---

#### Features

Simple counts for each user-merchant pair of actions (at ___merchant level___ instead of cat/brand/item level)

##### User info

In [28]:
user_info_df = pd.read_csv('./processes_data/user-info-df-12312017.csv')
user_info_df.head(5)

,user_id,age_range,gender
0,376517,6,1
1,234512,5,0
2,344532,5,0
3,186135,5,0
4,30230,5,0


##### User log

In [45]:
user_action_df = pd.read_csv('./data/data_format1/user_log_format1.csv', usecols=[0, 3, 6])

In [46]:
user_action_df.head(3)

,user_id,seller_id,action_type
0,328862,2882,0
1,328862,2882,0
2,328862,2882,0


---

#### Feature engineering

For each user-merchant pair, there may be more than one action. For now I just count each action type

<div class="mark">
Note: 0 is for click, 1 is for add-to-cart, 2 is for purchase and 3 is for add-to-favourite.</div><i class="fa fa-lightbulb-o "></i>

In [71]:
feature_df = train_df[train_df.columns[:2]].copy()
feature_df.set_index(['user_id', 'merchant_id'], inplace=True)
feature_df.head(2)

Empty DataFrame
Columns: []
Index: [(34176, 3906), (34176, 121)]

In [86]:
count_cols = ['click_count', 'cart_count', 'purchase_count', 'fav_count']

In [87]:
for col in count_cols:
    feature_df[col] = 0
    feature_df[col] = feature_df[col].astype(int)

In [88]:
feature_df.head(2)

click_count  cart_count  purchase_count  fav_count
user_id merchant_id                                                    
34176   3906                   0           0               0          0
        121                    0           0               0          0

In [89]:
for (uid, mid), um_df in user_action_df.groupby(['user_id', 'seller_id']):
    action_count = um_df.action_type.value_counts()
    for idx in range(4):
        if idx in action_count.index.values:
            feature_df.loc[(uid, mid), count_cols[idx]] = int(action_count[idx])

KeyboardInterrupt: 

In [ ]:
feature_df.head()